In [ ]:
import pandas as pd
xrefs_file="../ontology/xref-all.tsv"
obsoletes_file="../ontology/obsoletes_ordo.tsv"
df_xrefs=pd.read_csv(xrefs_file,sep="\t")
df_ordo=pd.read_csv(obsoletes_file,sep="\t")
df_ordo['deprecated']=True
df_ordo.head()

In [ ]:
df_xrefs[~df_xrefs['?deprecated'].isna()].head()

In [ ]:
df_xrefs_ordo=df_xrefs[(df_xrefs['?prefix']=="Orphanet")]
df_xrefs_ordo = df_xrefs_ordo.merge(df_ordo[['?curie','deprecated']], left_on=['?xref'], right_on=['?curie'])
print(len(df_xrefs_ordo))
df_xrefs_ordo.head()

In [ ]:
df_xrefs_ordo.to_csv("../ontology/ordo_deprecated.tsv", sep="\t")

# SSSOM Files

In [ ]:
import pandas as pd
from pathlib import Path
from argparse import ArgumentParser

def get_dataframe(tsv):
    try:
        df = pd.read_csv(tsv,sep="\t", comment="#")
        df["source"]=Path(tsv).stem
        return df
    except pd.errors.EmptyDataError:
        print("WARNING! ", tsv, " is empty and has been skipped.")

sssom_tsv_path = "/Users/matentzn/ws/mondo/src/ontology/mappings/ordo-omim.sssom.tsv"
df = get_dataframe(sssom_tsv_path)

sssom_files = {}

for index, row in df.iterrows():
    object_id = row["object_id"]
    prefix = object_id.split(":")[0]
    if prefix not in sssom_files:
        sssom_files[prefix] = []
    sssom_files[prefix].append(row)

for prefix in sssom_files:
    if
    print(prefix)

print(df.head())


In [ ]:
ps = "https://omim.org/phenotypicSeriesTitles/all?format=tsv"
mim2gene = "https://omim.org/static/omim/data/mim2gene.txt"
mimTitles = "https://data.omim.org/downloads/Wi95pIjsRfqt4ioi9rEJNQ/mimTitles.txt"
genemap2 = "https://data.omim.org/downloads/Wi95pIjsRfqt4ioi9rEJNQ/genemap2.txt"
morbidmap = "https://data.omim.org/downloads/Wi95pIjsRfqt4ioi9rEJNQ/morbidmap.txt"

In [ ]:
import pandas as pd
df_ps = pd.read_csv(ps,sep="\t")

In [ ]:
df_ps.head()

In [ ]:
import pandas as pd
import yaml
from datetime import datetime

import os
from sssom.datamodel_util import read_pandas, read_metadata
mapping_dir="../ontology/mappings/"
sssom_file=os.path.join(mapping_dir,"mondo.sssom.tsv")
metadata_file=os.path.join(mapping_dir,"mondo.sssom.yml")

meta, curie_map=read_metadata(metadata_file)
meta["mapping_date"]=datetime.now().strftime("%Y-%m-%d")
df=read_pandas(sssom_file)
df["match_type"]="HumanCurated"
df.head()

In [ ]:
curie_map

In [ ]:
subject_prefixes=set(df['subject_id'].str.split(':', 1, expand=True)[0])
object_prefixes=set(df['object_id'].str.split(':', 1, expand=True)[0])
#object_prefixes.remove("MONDO")
relations=set(df['predicate_id'])

In [ ]:
relations

In [ ]:
from sssom.parsers import from_dataframe
from sssom.writers import write_tsv
import re
import logging

def reconcile_curie(pre_orig):
    if re.match(r".*__[0-9]$", pre_orig):
        print(f"{pre_orig} non standard prefix, reconciling")
        pre = pre_orig[:-3]
    else:
        pre = pre_orig
    return pre


def replace_temporary_prefixes(prefixes, df):
    for pre_orig in prefixes:
        pre = reconcile_curie(pre_orig)
        if pre!=pre_orig:
            print(f"Replacing temporary curie {pre_orig} with {pre}")
            logging.warning(f"Replacing temporary curie {pre_orig} with {pre}")
            df['object_id'] = df['object_id'].str.replace(pre_orig+":",pre+":")
            df['subject_id'] = df['subject_id'].str.replace(pre_orig+":",pre+":")
    return df

df = replace_temporary_prefixes(subject_prefixes, df)
df = replace_temporary_prefixes(object_prefixes, df)

for pre_subj in subject_prefixes:
    for pre_obj in object_prefixes:
        for rel in relations:
            relpre=rel.split(":")[0]
            relppost=rel.split(":")[1]
            sssom_file=os.path.join(mapping_dir,f"{pre_subj.lower()}_{relppost.lower()}_{pre_obj.lower()}.sssom.tsv")
    
            dfs=df[(df['subject_id'].str.startswith(pre_sub+":")) 
                   & (df['predicate_id'] == rel) 
                   & (df['object_id'].str.startswith(pre_obj+":"))]
            if pre_subj in curie_map and pre_obj in curie_map and len(dfs)>0:
                cm = {pre_sub: curie_map[pre_sub], pre_obj: curie_map[pre_obj], relpre: curie_map[relpre]}
                msdf=from_dataframe(dfs,curie_map=cm,meta=meta)
                write_tsv(msdf=msdf,filename=sssom_file)
                print(f"Writing {sssom_file} complete!")
            else:
                print(f"Not creating {sssom_file} because there is a missing prefix ({pre_subj}, {pre_obj}), or no matches ({len(dfs)} matches found)")
    
